In [1]:
import os 

os.getcwd()
dir_ = 'C://Users//Domin/Downloads'
os.chdir(dir_)

os.getcwd()
import pandas as pd
import numpy 
import torch
import torch.nn as nn
import matplotlib.pyplot as plt


In [2]:
data = pd.read_csv('breastcancer.csv')

#lets do a little exploratory analysis 

data.dropna(axis = 1)
data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [3]:


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder 
labeler = LabelEncoder()
y = data['diagnosis'] 
x = data.drop(['diagnosis', 'id'], axis = 1) 
x = x[['radius_mean', 'texture_mean', 'area_mean', 'smoothness_mean', 'concavity_mean', 'radius_worst']]   


scaler = StandardScaler()


In [4]:
x

,radius_mean,texture_mean,area_mean,smoothness_mean,concavity_mean,radius_worst
0,17.99,10.38,1001.0,0.11840,0.30010,25.380
1,20.57,17.77,1326.0,0.08474,0.08690,24.990
2,19.69,21.25,1203.0,0.10960,0.19740,23.570
3,11.42,20.38,386.1,0.14250,0.24140,14.910
4,20.29,14.34,1297.0,0.10030,0.19800,22.540
...,...,...,...,...,...,...
564,21.56,22.39,1479.0,0.11100,0.24390,25.450
565,20.13,28.25,1261.0,0.09780,0.14400,23.690
566,16.60,28.08,858.1,0.08455,0.09251,18.980
567,20.60,29.33,1265.0,0.11780,0.35140,25.740


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 24)

y_train = labeler.fit_transform(y_train)
y_test = labeler.fit_transform(y_test)



In [6]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)



X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [7]:
#modified version - borrowed some code and modified for the functions


class NeuralNetworkClassificationModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(NeuralNetworkClassificationModel,self).__init__()
        self.input_layer    = nn.Linear(input_dim,200)
        self.hidden_layer1  = nn.Linear(200,64)
        self.output_layer   = nn.Linear(64,output_dim)
        self.relu = nn.ReLU()
    
    
    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out
    


In [8]:
# input_dim = 4 because we have 4 inputs namely sepal_length,sepal_width,petal_length,petal_width
# output_dim = 3 because we have namely 3 categories setosa,versicolor and virginica
input_dim  = 6 
output_dim = 2
model = NeuralNetworkClassificationModel(input_dim,output_dim)

In [9]:
# creating our optimizer and loss function object
learning_rate = 0.0001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [10]:
import numpy as np 

In [11]:
def train_network(model,optimizer,criterion,X_train,y_train,X_test,y_test,num_epochs,train_losses,test_losses):
    for epoch in range(num_epochs):
        #clear out the gradients from the last step loss.backward()
        optimizer.zero_grad()
        
        #forward feed
        output_train = model(X_train)

        #calculate the loss
        loss_train = criterion(output_train, y_train)
        


        #backward propagation: calculate gradients
        loss_train.backward()

        #update the weights
        optimizer.step()

        
        output_test = model(X_test)
        loss_test = criterion(output_test,y_test)

        train_losses[epoch] = loss_train.item()
        test_losses[epoch] = loss_test.item()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss_train.item():.4f}, Test Loss: {loss_test.item():.4f}")

In [12]:
num_epochs = 1000
train_losses = np.zeros(num_epochs)
test_losses  = np.zeros(num_epochs)

In [13]:
train_network(model,optimizer,criterion,X_train,y_train,X_test,y_test,num_epochs,train_losses,test_losses)

Epoch 50/1000, Train Loss: 0.4126, Test Loss: 0.4197
Epoch 100/1000, Train Loss: 0.2640, Test Loss: 0.2702
Epoch 150/1000, Train Loss: 0.1904, Test Loss: 0.1948
Epoch 200/1000, Train Loss: 0.1535, Test Loss: 0.1588
Epoch 250/1000, Train Loss: 0.1320, Test Loss: 0.1394
Epoch 300/1000, Train Loss: 0.1181, Test Loss: 0.1280
Epoch 350/1000, Train Loss: 0.1086, Test Loss: 0.1211
Epoch 400/1000, Train Loss: 0.1016, Test Loss: 0.1165
Epoch 450/1000, Train Loss: 0.0961, Test Loss: 0.1134
Epoch 500/1000, Train Loss: 0.0915, Test Loss: 0.1110
Epoch 550/1000, Train Loss: 0.0876, Test Loss: 0.1091
Epoch 600/1000, Train Loss: 0.0842, Test Loss: 0.1076
Epoch 650/1000, Train Loss: 0.0811, Test Loss: 0.1063
Epoch 700/1000, Train Loss: 0.0782, Test Loss: 0.1051
Epoch 750/1000, Train Loss: 0.0756, Test Loss: 0.1040
Epoch 800/1000, Train Loss: 0.0732, Test Loss: 0.1031
Epoch 850/1000, Train Loss: 0.0709, Test Loss: 0.1022
Epoch 900/1000, Train Loss: 0.0688, Test Loss: 0.1014
Epoch 950/1000, Train Loss: 0

In [14]:
predictions_train = []
predictions_test =  []
with torch.no_grad():
    predictions_train = model(X_train)
    predictions_test = model(X_test)

In [15]:
def get_accuracy_multiclass(pred_arr,original_arr):
    if len(pred_arr)!=len(original_arr):
        return False
    pred_arr = pred_arr.numpy()
    original_arr = original_arr.numpy()
    final_pred= []
    # we will get something like this in the pred_arr [32.1680,12.9350,-58.4877]
    # so will be taking the index of that argument which has the highest value here 32.1680 which corresponds to 0th index
    for i in range(len(pred_arr)):
        final_pred.append(np.argmax(pred_arr[i]))
    final_pred = np.array(final_pred)
    count = 0
    #here we are doing a simple comparison between the predicted_arr and the original_arr to get the final accuracy
    for i in range(len(original_arr)):
        if final_pred[i] == original_arr[i]:
            count+=1
    return count/len(final_pred)

In [16]:
train_acc = get_accuracy_multiclass(predictions_train,y_train)
test_acc  = get_accuracy_multiclass(predictions_test,y_test)

In [17]:
print(f"Training Accuracy: {round(train_acc*100,3)}")
print(f"Test Accuracy: {round(test_acc*100,3)}")

Training Accuracy: 98.022
Test Accuracy: 96.491
